# Overview of key results so far

In [19]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


# Introduction

In this research project, we are interested in knowing how the future southern ocean carbon sink will be impacted by the interaction of 2 main drivers:

- evolution of atmospheric O$_3$ concentration in the Southern Ocean 
    - 3 different scenarios:
    - no ozone hole ("Fixed Ozone" or "Ozone 1950")
    - ozone hole opens and then recovers ("Hist. Ozone")
    - ozone hole opens around 1990 and then never recovers ("1990 Ozone")
- evolution of atmospheric GHG concentrations
    - 2 scenarios: 
    - SSP1-2.6 (LOW CARBON)
    - SSP 3-7.0 (HIGH CARBON)
    
We force a model (currently PlankTOM12, with a small adaptation to prevent GOC "explosions", see https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/MODPROC_ROBOT/MultiModelMonitor/SANITY_MONITOR.ipynb) with atmospheric forcing from 6 scenarios (combination of the above) of the UKESM1. This forcing has been adapted to work with NEMO (see https://nbviewer.org/github/tjarnikova/SOZONE/tree/master/MO_pipeline/ and https://docs.google.com/document/d/1sK9lCSdDUcnPzPrVHjrkluf2xh09VzTUMCh7yDlOkDk/edit). 

Here I'm keeping track of main results and considerations so far (with pictures!)

# A) Comparison of 3 types of wind forcing

The 3 types of wind forcing we can use to force a NEMO ocean model:

- NCEP- NCEP/NCAR reanalysis (https://gitlab.com/env_greenocean/greenocean_documentation/-/blob/main/HowToCreateNCEPForcingFiles.pdf)
    - /gpfs/data/greenocean/software/products/NCEPForcingData
    - uses CLIO bulk formulation https://www.nemo-ocean.eu/doc/node49.html
    - known wind issues, see docs on gitlab
    - essentially observational
    
- ERA5
    - /gpfs/data/greenocean/software/products/ERA5Forcing
    - https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5
    - uses CORE bulk formulation 
    - essentially observational
    - issues around year 1979

- MetOffice
    - processed data is in: /gpfs/data/greenocean/software/resources/MetProcessed/MET_forcing/
    - output of UKESM1 (not observational, has internal weather)
    
The 3 types of forcing are broadly comparable:
https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/prelimEvalForcing/wind_climatology_maps.ipynb

But there are important differences:


In [20]:
# from https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/prelimEvalForcing/wind_climatology_maps.ipynb
pdir = '../uploadPics/windzonal.jpg'
from IPython.display import Image
Image(url= pdir, width=400, height=400)

Basically, in general, NCEP starts out at a lower windspeed than UKESM1/ERA and experiences a stronger increase in winds over 1950-2020 in both summer and winter. Winter winds have increased only slightly in ERA/UKESM1, the increase in NCEp has been stronger. Net magnitudes of wind are also lower.

Importantly/unsettlingly, the 3 different wind forcing products lead to differnt estimations of the total air-sea CO2 flux, both in the total domain and in the southern ocean specifically. A few key points from a summary figure of 4 runs (NCEP, ERA, MET with GOC brakes, MET without GOC brakes):

1) The NCEP run (GAO1, blue, using CLIO) has somewhat lower uptake than the others, both globally and in SO specifically. 
    - this lower uptake is despite lower winds overall: high winds in SO should lead to lower carbon sink. 
    - on the other hand, the NCEP wind product is the only one that sees an increase in SO winds in winter, which could be the cause of a slowdown
    - or it could have something to do with using CLIO instead of CORE.
    
2) The other 3 runs, using CORE and comparable though independent wind products, tend to have similar estimates of CO2 flux, both globally and in the SO. 
    - the "weirdness" in the first 30 years or so of the air-sea co2 flux in the yellow met office run is partly because this run is not spun up - will be fixed. after 30 years or so it seems to settle into giving very similar results to the two ERA5 runs.
    
3) We do see an effect of the GOC brakes (dotted black line vs grey line). In the 2010s, when the original DW_WE43 run has a GOC and EXP exlosion (goc and export increase unstably), the TJ_WE43 run has lower sink. Ie, less export > more outgassing > lower sink. 

My guess is that the bulk formulation, more than the wind speed magnitudes, determines the differences in flux here. However it means that important features (eg. '90s slowdown in SO CO2 uptake') may be a feature more of bulk formulation than windspeed magnitudes (or some combination of the two). 

In [21]:
#https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/MODPROC_ROBOT/MultiModelMonitor/CFLX_MONITOR.ipynb
pdir = '../uploadPics/co2flux_era_met_ncep.jpg'
from IPython.display import Image
Image(url= pdir, width=700, height=400)


We can also look a bit more in-depth at how the MET office winds compare with the ERA5 product. TL;DR: really quite well. 

I did some high-detail histograms here:
https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/windAnalyis/MonthlyERAMetHistograms.ipynb

Here is a by-latitude comparison of the winds in ERA5 and Met Office:
https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/windAnalyis/LatitudinalWinds.ipynb

A good summary figure from it is the following set of latitudinally-divided time-progressing histograms of summer wind speed for both ERA and MET/UKESM1:

Obvious takeaways:
- ERA and MO agree really pretty well, but best in midlatitudes -64 to - 55 say?
- MO underestimates winds in high latitudes (near land/over ice )? but slightly overestimates in lower latitudes
- the most obvious speedup of summer winds in both datasets is in bands something like -65 to -50 - a good place to look for results. 

In [22]:
#https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/windAnalyis/wind_violin.ipynb
pdir = '../uploadPics/lat_summer_winds_era_met.jpg'
from IPython.display import Image
Image(url= pdir, width=500, height=400)

# B) Time trends in the UKESM1 forcing:

I did a fairly in-depth analysis of how the winds shift in summer and winter. As expected (why?) we expect bigger shifts in summer than in winter. 
https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/windAnalyis/wind_violin.ipynb

Satisfyingly, the following (10-year running mean averaged, for clarity) analysis of the summer and winter winds shows what we expect to see: 

- increases in summer are bigger than in winter (expected)
- high co2 scenarios always have higher wind speeds than lower co2 scenarios (expected)
- in the ozone recovery scenario, the 2nd half of 21st century winds decrease again, but only under low carbon scen. (semi-expected, cool!)

In [23]:
#https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/windAnalyis/wind_violin.ipynb
pdir = '../uploadPics/wspd_10yrmean.jpg'
from IPython.display import Image
Image(url= pdir, width=500, height=400)

# C) Evolution of the SO carbon sink based on these 6 wind forcings

So far I've run these 6 scenarios once, using PlankTOM12, from 1948 (a restart from DW_WE43). This version of the model seems to have a better ice model (says Dave), I added GOC brakes because things tend to go unstable otherwise. Here is a summary plot of the air-sea co2 flux and pco2 for the 6 runs. A more in-depth subdivision, with decadal and seasonal trends, is here: https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/MO_RESULTS/SeasonalCflxFullRunset.ipynb

Key takeaways: 
- the GHG signal completely overwhelms the O$_3$ signal - we see results between GHG scenarios but not between the O3 ones. differences between wind forcings seem too subtle?
- there is a saturation of the global carbon sink in the latter half of the 21st century on a global level, under SSP3-7.0

In [24]:
#https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/MO_RESULTS/SeasonalCflxFullRunset.ipynb
pdir = '../uploadPics/CFLX_SA_runs.jpg'
from IPython.display import Image
Image(url= pdir, width=900, height=400)

So, why is there no response to differences in wind in the CO2 flux? I still don't really know. The effect size may be very small?  

I made some videos:
analysis here :https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/MO_RESULTS/LatitudinalCflxWinds.ipynb
and here: https://nbviewer.org/github/tjarnikova/SOZONE/blob/master/MO_RESULTS/plotsForMovies.ipynb

- https://drive.google.com/drive/folders/1L9bRD2mZreUsLWG7QJGnpRXn8NqU8UWL

It is very hard to tell differences between the wind behaviours of the different scenarios. Looking at latitudinal trends (first link) also doesn't tell us very much - though there are distinct patterns that I cannot explain. From part A, we know that the biggest changes in windspeed are between -65 and -50. 
